In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio

In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8051"

In [4]:
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

def research_messages_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle research messages ", payload, connection_id)

research_message_listener = {
    "handler": research_messages_handler,
    "topic": "basicmessages"
}



In [5]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending coro=<AriesAgentController.listen_webhooks() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py:42>>

In [6]:
agent_controller.register_listeners([research_message_listener], defaults=True)

In [7]:
#Paste in invitation from data_owner agent
invite = {'connection_id': 'fe0b1101-649c-4e23-9084-9bc6dd5e6150', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '1c8f447e-271f-43cf-be83-2f647d5ebcb2', 'recipientKeys': ['C5gih4nhTrQUUcrQ4bWxNiz5R6Jbw1WJ8GrcNEyT2cTE'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Bob'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWM4ZjQ0N2UtMjcxZi00M2NmLWJlODMtMmY2NDdkNWViY2IyIiwgInJlY2lwaWVudEtleXMiOiBbIkM1Z2loNG5oVHJRVVVjclE0Yld4Tml6NVI2SmJ3MVdKOEdyY05FeVQyY1RFIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE5Mi4xNjguNjUuMzo4MDIwIiwgImxhYmVsIjogIkJvYiJ9'}

In [8]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
print(response)
# Print out accepted Invite and Researcher ID
print("Connection", response)
researcher_id = response["connection_id"]


ACCEPTED CONNECTION -  {'initiator': 'external', 'updated_at': '2020-06-25 19:25:03.385544Z', 'request_id': 'ec153399-f3d3-4109-b48a-3e626edd6c85', 'invitation_mode': 'once', 'routing_state': 'none', 'connection_id': '846daefe-83e6-4139-a236-31f142724420', 'accept': 'manual', 'created_at': '2020-06-25 19:25:03.368209Z', 'state': 'request', 'invitation_key': 'C5gih4nhTrQUUcrQ4bWxNiz5R6Jbw1WJ8GrcNEyT2cTE', 'their_label': 'Bob', 'my_did': '5ZZDzLNPUxYtbML35bd89N'}
{'initiator': 'external', 'updated_at': '2020-06-25 19:25:03.385544Z', 'request_id': 'ec153399-f3d3-4109-b48a-3e626edd6c85', 'invitation_mode': 'once', 'routing_state': 'none', 'connection_id': '846daefe-83e6-4139-a236-31f142724420', 'accept': 'manual', 'created_at': '2020-06-25 19:25:03.368209Z', 'state': 'request', 'invitation_key': 'C5gih4nhTrQUUcrQ4bWxNiz5R6Jbw1WJ8GrcNEyT2cTE', 'their_label': 'Bob', 'my_did': '5ZZDzLNPUxYtbML35bd89N'}
Connection {'initiator': 'external', 'updated_at': '2020-06-25 19:25:03.385544Z', 'request_

In [9]:
# Do a trust ping to ensure connection is accepted by data_owner
trust_ping = await agent_controller.connections.trust_ping(researcher_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA")
print(trust_ping)   

TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA
{'thread_id': '0785039a-2ee9-4cf9-b677-0ed25db34442'}


In [14]:
# Print connection list
connection = await agent_controller.connections.get_connection(researcher_id)
print("RESEARCH AGENT CONNECTION")
print(connection)

RESEARCH AGENT CONNECTION
{'initiator': 'external', 'updated_at': '2020-06-25 19:25:13.152328Z', 'request_id': 'ec153399-f3d3-4109-b48a-3e626edd6c85', 'invitation_mode': 'once', 'routing_state': 'none', 'their_did': 'EKFusCHGwjMWdpDr3Ti3cC', 'connection_id': '846daefe-83e6-4139-a236-31f142724420', 'accept': 'manual', 'created_at': '2020-06-25 19:25:03.368209Z', 'state': 'active', 'invitation_key': 'C5gih4nhTrQUUcrQ4bWxNiz5R6Jbw1WJ8GrcNEyT2cTE', 'their_label': 'Bob', 'my_did': '5ZZDzLNPUxYtbML35bd89N'}


In [19]:
#send some basic messages
message = await agent_controller.connections.send_message(researcher_id,"hello from researcher world!")
print("BASIC MESSAGE - RESEARCH -> DATA")
print(message)

BASIC MESSAGE - RESEARCH -> DATA
{}


In [10]:
import torch
import syft as sy
from syft.serde import deserialize
from syft.serde import serialize
from syft.workers.base import BaseWorker


class AriesWorker(BaseWorker):
    def _send_msg(self, message: bin, location: BaseWorker) -> bin:
      return self._recv_msg(message)

    def _recv_msg(self, message: bin) -> bin:
      json_message = deserialize(message)
      print(json_message)
      result = self.recv_msg(message)
      result = deserialize(result)
      print(result)
      
      message = asyncio.get_event_loop().create_task(agent_controller.connections.send_message(researcher_id, str(result) ))
      print(message)
      return result

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/opt/conda/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [11]:
hook = sy.TorchHook(torch)
alice = AriesWorker(hook, id="claire")

In [12]:
def research_syft_handler(payload):
    connection_id = payload["connection_id"]
    alice._recv_msg(serialize(payload["content"]))
    print("Handle research messages ", payload, connection_id)

research_syft_listener = {
    "handler": research_syft_handler,
    "topic": "basicmessages"
}


In [13]:
agent_controller.register_listeners([research_syft_listener], defaults=False)

NameError: name 'GRID' is not defined